# data

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
from pathlib import Path
from pymicro.crystal.ebsd import OimScan

filename = "Ti_ebsd_demo.h5"
filepath = Path(filename).absolute()
assert filepath.exists()

scan = OimScan.from_file(str(filepath))

del filename, filepath

In /home/esrf/casagran/miniconda3/envs/pymicro-esrf-slurm-intel/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/esrf/casagran/miniconda3/envs/pymicro-esrf-slurm-intel/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/esrf/casagran/miniconda3/envs/pymicro-esrf-slurm-intel/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/esrf/casagran/miniconda3/envs/pymicro-esrf-slurm-intel/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_n

Ti_ebsd_demo .h5
reading EBSD scan Pattern cropped cleaned-Dil from file /home/esrf/casagran/pymicro/dev/joao-ebsd-segment-grains/Ti_ebsd_demo.h5


# linear algo

In [3]:
%autoreload 2
import numpy as np
from skimage.future.graph import RAG
import itertools
from scipy.spatial.distance import cdist
import progressbar
from pymicro.crystal.microstructure import Orientation
from pymicro.crystal.lattice import Symmetry
from skimage.future.graph import cut_threshold

def skimage_method(scan_):
    shape = scan_.iq.shape
    size = shape[0] * shape[1]
    # each pixel has its own label
    labels = np.arange(size).reshape(shape)
    rag = RAG(label_image=labels, connectivity=1) 
    for node in rag.nodes:
        rag.nodes[node]['labels'] = [node]
    pixel_indices = np.array(list(itertools.product(range(shape[0]), range(shape[1]))))
    pixel_position_distances = cdist(pixel_indices, pixel_indices, metric='cityblock')
    pixels_to_get = np.stack(np.where(pixel_position_distances == 1)).T
    pixels_to_get = pixels_to_get[pixels_to_get[:, 0] < pixels_to_get[:, 1]]  # get rid of duplicates
    for pix1_idx, pix2_idx in progressbar.progressbar(pixels_to_get):
        pixel1_pos = tuple(pixel_indices[pix1_idx])
        pixel2_pos = tuple(pixel_indices[pix2_idx])
        euler1 = scan_.euler[pixel1_pos]
        euler2 = scan_.euler[pixel2_pos]
        o1 = Orientation.from_euler(np.degrees(euler1))
        o2 = Orientation.from_euler(np.degrees(euler2))
        disor = o1.disorientation(o2, crystal_structure=Symmetry.hexagonal)  # angles (weights) are in radians
        angle = disor[0]
        lab1 = labels[pixel1_pos]
        lab2 = labels[pixel2_pos]
        rag.add_edge(lab1, lab2, dict(weight=angle))
    threshold_deg = 5 
    threshold_rad = np.radians(threshold_deg)
    return cut_threshold(labels, rag, threshold_rad)    

In [4]:
%%time
grain_ids_linear = skimage_method(scan)

N/A% (0 of 20200) |                      | Elapsed Time: 0:00:00 ETA:  --:--:--/home/esrf/casagran/pymicro/pymicro/crystal/microstructure.py:335: RuntimeWarning: invalid value encountered in true_divide
  n /= np.sqrt((delta[1, 2] - delta[2, 1]) ** 2 +
/home/esrf/casagran/pymicro/pymicro/crystal/microstructure.py:373: RuntimeWarning: invalid value encountered in arccos
  omega = np.arccos(cw)
100% (20200 of 20200) |##################| Elapsed Time: 0:01:02 Time:  0:01:02


CPU times: user 1min 3s, sys: 794 ms, total: 1min 4s
Wall time: 1min 3s


# parallel algo

In [72]:
%%writefile skimage_method_parallel.py

import numpy as np
from skimage.future.graph import RAG
import itertools
from scipy.spatial.distance import cdist
import progressbar
from pymicro.crystal.microstructure import Orientation
from pymicro.crystal.lattice import Symmetry
from skimage.future.graph import cut_threshold
from multiprocessing import Pool
import os

import contextlib

    
    
def worker(lab1, lab2, euler1, euler2):
    o1 = Orientation.from_euler(np.degrees(euler1))
    o2 = Orientation.from_euler(np.degrees(euler2))
    disor = o1.disorientation(o2, crystal_structure=Symmetry.hexagonal)  # angles (weights) are in radians
    return (lab1, lab2, disor[0])
    

def skimage_method_parallel(scan_, threshold_deg, nprocs):
    shape = scan_.iq.shape
    size = shape[0] * shape[1]
    # each pixel has its own label
    labels = np.arange(size).reshape(shape)
    rag = RAG(label_image=labels, connectivity=1) 
    for node in rag.nodes:
        rag.nodes[node]['labels'] = [node]
    pixel_indices = np.array(list(itertools.product(range(shape[0]), range(shape[1]))))
    pixel_position_distances = cdist(pixel_indices, pixel_indices, metric='cityblock')
    pixels_to_get = np.stack(np.where(pixel_position_distances == 1)).T
    pixels_to_get = pixels_to_get[pixels_to_get[:, 0] < pixels_to_get[:, 1]]  # get rid of duplicates
    args_list = []
    for pix1_idx, pix2_idx in pixels_to_get:
        pixel1_pos = tuple(pixel_indices[pix1_idx])
        pixel2_pos = tuple(pixel_indices[pix2_idx])
        euler1 = scan_.euler[pixel1_pos]
        euler2 = scan_.euler[pixel2_pos]
        lab1 = labels[pixel1_pos]
        lab2 = labels[pixel2_pos]
        args_list.append((lab1, lab2, euler1, euler2))
    
    with contextlib.closing(Pool(processes=nprocs,)) as pool:
        edges = pool.starmap_async(worker, args_list)
    pool.join()
    assert edges.ready()
    assert edges.successful()
    edges = [e for e in edges.get()]
    
    for lab1, lab2, angle in edges:
        rag.add_edge(lab1, lab2, dict(weight=angle))
        
     
    threshold_rad = np.radians(threshold_deg)
    
    return cut_threshold(labels, rag, threshold_rad)    

Overwriting skimage_method_parallel.py


In [73]:
del skimage_method_parallel

In [74]:
%%time
%autoreload
from skimage_method_parallel import skimage_method_parallel
grain_ids_linear = skimage_method_parallel(scan, 5, nprocs=8)

/home/esrf/casagran/pymicro/pymicro/crystal/microstructure.py:335: RuntimeWarning: invalid value encountered in true_divide
  n /= np.sqrt((delta[1, 2] - delta[2, 1]) ** 2 +
/home/esrf/casagran/pymicro/pymicro/crystal/microstructure.py:335: RuntimeWarning: invalid value encountered in true_divide
  n /= np.sqrt((delta[1, 2] - delta[2, 1]) ** 2 +
/home/esrf/casagran/pymicro/pymicro/crystal/microstructure.py:335: RuntimeWarning: invalid value encountered in true_divide
  n /= np.sqrt((delta[1, 2] - delta[2, 1]) ** 2 +
/home/esrf/casagran/pymicro/pymicro/crystal/microstructure.py:373: RuntimeWarning: invalid value encountered in arccos
  omega = np.arccos(cw)
/home/esrf/casagran/pymicro/pymicro/crystal/microstructure.py:335: RuntimeWarning: invalid value encountered in true_divide
  n /= np.sqrt((delta[1, 2] - delta[2, 1]) ** 2 +
/home/esrf/casagran/pymicro/pymicro/crystal/microstructure.py:373: RuntimeWarning: invalid value encountered in arccos
  omega = np.arccos(cw)
/home/esrf/casagra

CPU times: user 1.02 s, sys: 296 ms, total: 1.31 s
Wall time: 8.37 s


In [75]:
%%time
%autoreload
from skimage_method_parallel import skimage_method_parallel
grain_ids_linear = skimage_method_parallel(scan, 5, nprocs=16)

/home/esrf/casagran/pymicro/pymicro/crystal/microstructure.py:335: RuntimeWarning: invalid value encountered in true_divide
  n /= np.sqrt((delta[1, 2] - delta[2, 1]) ** 2 +
/home/esrf/casagran/pymicro/pymicro/crystal/microstructure.py:335: RuntimeWarning: invalid value encountered in true_divide
  n /= np.sqrt((delta[1, 2] - delta[2, 1]) ** 2 +
/home/esrf/casagran/pymicro/pymicro/crystal/microstructure.py:373: RuntimeWarning: invalid value encountered in arccos
  omega = np.arccos(cw)
/home/esrf/casagran/pymicro/pymicro/crystal/microstructure.py:335: RuntimeWarning: invalid value encountered in true_divide
  n /= np.sqrt((delta[1, 2] - delta[2, 1]) ** 2 +
/home/esrf/casagran/pymicro/pymicro/crystal/microstructure.py:335: RuntimeWarning: invalid value encountered in true_divide
  n /= np.sqrt((delta[1, 2] - delta[2, 1]) ** 2 +
/home/esrf/casagran/pymicro/pymicro/crystal/microstructure.py:335: RuntimeWarning: invalid value encountered in true_divide
  n /= np.sqrt((delta[1, 2] - delta[2

CPU times: user 1.02 s, sys: 358 ms, total: 1.38 s
Wall time: 5.01 s
